### Configuración y Dependencias

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()
api_key = os.getenv("API_KEY")

# *** Aqui se cambia el PDF ***
PDF_PATH = "OLLAMA.pdf"

print(" Dependencias cargadas")
print(f" PDF seleccionado: {PDF_PATH}")

✓ Dependencias cargadas
✓ PDF seleccionado: OLLAMA.pdf


### Cargar Documento PDF

In [ ]:
def upload_pdf(url: str):
    try:
        loader = PyPDFLoader(url)
        loader = loader.lazy_load()

        text = ""
        for page in loader:
            text += page.page_content + "\n"

        print(f" PDF cargado: {len(text)} caracteres")
        return text
    except Exception as e:
        print(f" Error: {e}")
        return ""

raw_text = upload_pdf(PDF_PATH)

✓ PDF cargado: 16971 caracteres


### Experimentación: Diferentes Configuraciones de Text Splitter

Prueba diferentes valores de `chunk_size` y `chunk_overlap` para ver cómo afectan los resultados.

In [ ]:
# Configuración 1: PEQUEÑO (más chunks, menos contexto)
def text_splitter_small(text):
    splitter = CharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separator="\n"
    )
    return splitter.create_documents([text])

# Configuración 2: MEDIANO (equilibrio)
def text_splitter_medium(text):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    return splitter.create_documents([text])

# Configuración 3: GRANDE (menos chunks, más contexto)
def text_splitter_large(text):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    return splitter.create_documents([text])

# USAR LA CONFIGURACIÓN QUE PREFIERAS
chunks = text_splitter_small(raw_text)  # ← Cambia a small, medium o large
print(f"Chunks creados: {len(chunks)}")

✓ Chunks creados: 39


### Experimentación: Diferentes Modelos de Embedding

In [ ]:
# Opción 1: nomic-embed-text (768D, rápido)
# embedding = OllamaEmbeddings(model="nomic-embed-text")

# Opción 2: mxbai-embed-large (1024D, más preciso) - PREDETERMINADO
embedding = OllamaEmbeddings(model="mxbai-embed-large:latest")

# Opción 3: all-minilm (384D, ultraligero)
# embedding = OllamaEmbeddings(model="all-minilm:latest")

print("Modelo de embedding seleccionado")

✓ Modelo de embedding seleccionado


### Base de Datos Vectorial (Chroma)

In [ ]:
def get_vector_store(name_collection: str):
    vector_store = Chroma(
        collection_name=name_collection,
        embedding_function=embedding,
        persist_directory="./vectorstore"
    )
    return vector_store

vector_store = get_vector_store("langchain")
print("Base de datos vectorial inicializada")

✓ Base de datos vectorial inicializada


### Agregar Documentos a la Base de Datos

In [ ]:
vector_store.add_documents(chunks)
print("Documentos agregados a la base de datos vectorial")

✓ Documentos agregados a la base de datos vectorial


### Función de Recuperación (Retrieval)

In [ ]:
def retrieval(input_user: str, k: int = 4):
    """Busca k documentos relevantes. Aumenta k para más contexto."""
    docs = vector_store.similarity_search(input_user, k=k)
    return docs

print("Función retrieval lista")

✓ Función retrieval lista


### Experimentación: Diferentes Prompts

Cambia el prompt para ver cómo afecta la calidad de las respuestas.

In [ ]:
# Prompt 1: SIMPLE (directo)
prompt_simple = PromptTemplate.from_template("""
Contexto: {contexto}
Pregunta: {input_user}
Respuesta:""")

# Prompt 2: CON INSTRUCCIONES (recomendado) - PREDETERMINADO
prompt_instructions = PromptTemplate.from_template("""
Eres un asistente experto. Responde basándote SOLO en el contexto.
Si la información no está disponible, responde: "No tengo esa información".

Contexto: {contexto}
Pregunta: {input_user}
Respuesta:""")

# Prompt 3: CON FUENTES (detalladp)
prompt_with_sources = PromptTemplate.from_template("""
Eres un asistente experto. Responde basándote en el contexto.
Al final, indica de dónde obtuviste la información.

Contexto: {contexto}
Pregunta: {input_user}

Respuesta (incluye atribución de fuentes):
""")

# USAR EL PROMPT QUE PREFIERAS
prompt = prompt_instructions  # ← Cambia a simple, instructions o with_sources
print("Prompt seleccionado")

✓ Prompt seleccionado


### Experimentación: Diferentes Modelos LLM

In [ ]:
def response(input_user: str, contexto: str, model_name: str = "gemini-2.0-flash", temperature: float = 0.7):
    """
    Genera respuesta usando LLM con streaming.
    
    Modelos disponibles:
    - gemini-2.0-flash (rápido, recomendado)
    - gemini-1.5-pro (más potente)
    - gemini-1.5-flash (alternativa)
    """
    llm = ChatGoogleGenerativeAI(
        api_key=api_key,
        model=model_name,
        temperature=temperature
    )

    for chunk in llm.stream(prompt.format(contexto=contexto, input_user=input_user)):
        yield chunk.content

print("Función response lista")

✓ Función response lista


### Loop Interactivo Mejorado

Prueba tu sistema RAG. Escribe `salir` para terminar.

In [16]:
while True:
    try:
        input_user = input("\nHuman: ").strip()
        
        if not input_user:
            continue
        
        if input_user.lower() in {"salir", "exit", "quit"}:
            break
        
        docs = retrieval(input_user, k=4)
        
        if docs:
            print("\nAssistant: ", end="", flush=True)
            for chunk in response(input_user, contexto=docs):
                print(chunk, end="", flush=True)
            print("\n")
    
    except KeyboardInterrupt:
        break
    except Exception as e:
        print(f"Error: {e}")


Assistant: Ollama esOllama es una herramienta de código abierto, escrita en GO, que permite ejecutar Grandes Modelos una herramienta de código abierto, escrita en GO, que permite ejecutar Grandes Modelos de Lenguajes (LLMs) de forma sencilla y local. Al ejecutar de Lenguajes (LLMs) de forma sencilla y local. Al ejecutar los modelos localmente, se mantiene la propiedad plena de los datos y se evitan posibles riesgos de filtraciones de información sensible. También ayuda a reducir la latencia y la los modelos localmente, se mantiene la propiedad plena de los datos y se evitan posibles riesgos de filtraciones de información sensible. También ayuda a reducir la latencia y la dependencia.

 dependencia.



### Análisis: Comparar Configuraciones

Ejecuta esta celda para probar rápidamente diferentes configuraciones y compararlas.

In [13]:
import pandas as pd
import time

# Preguntas de prueba
test_questions = [
    "¿Cuál es el tema principal del documento?",
    "¿Qué información importante hay?",
    "¿Puedes resumir esto?"
]

# Probar diferentes configuraciones
results = []

for question in test_questions:
    start_time = time.time()
    docs = retrieval(question, k=3)
    elapsed = time.time() - start_time
    
    results.append({
        "Pregunta": question[:50],
        "Documentos encontrados": len(docs),
        "Tiempo (s)": round(elapsed, 3),
        "Chars del primer doc": len(docs[0].page_content) if docs else 0
    })

df = pd.DataFrame(results)
print("\nComparación de Búsquedas:")
print(df.to_string(index=False))


Comparación de Búsquedas:
                                 Pregunta  Documentos encontrados  Tiempo (s)  Chars del primer doc
¿Cuál es el tema principal del documento?                       3       1.001                   478
         ¿Qué información importante hay?                       3       0.722                   495
                    ¿Puedes resumir esto?                       3       0.803                   498
